# Agent chat

In [2]:
import sys
sys.path.append('../')
from botcore.setup import trace_ai21
MODEL = trace_ai21('vechai')


from langchain.llms import BaseLLM
from langchain.agents import AgentType, initialize_agent
import sys
sys.path.append('../')
from botcore.chains.assess_usage import build_assess_usage_tool
from botcore.chains.pros_cons import build_pros_cons_tool


class AgentBot:

    def __init__(self, model: BaseLLM, memory):
        tools = self.load_tools(model, memory)
        self.agent = initialize_agent(tools, model,\
                                      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,\
                                      verbose=True, return_intermediate_steps=True)
        print("Agent is ready")
        
        
    def load_tools(self, model, memory):
        tools = [build_assess_usage_tool(model,memory),
                 build_pros_cons_tool(model,memory)]
        return tools

    

Enable tracing at vechai
AI21 ready


In [3]:
from botcore.utils.memory_utils import QAMemory
data = {
  "title": "Old phone",
  "features": [
    "What is the screen size? 2.8 inches",
    "What is the RAM size? 512 MB",
    "What is the storage capacity? 4 GB",
    "What is the battery capacity? 1500 mAh",
    "Is there any malfunction or defect? yes",
    "What is the current physical condition of the product? excellent",
    "Is the product still under warranty? yes"
  ]
}

ques = [i.split("?")[0] for i in data['features']]
ans = [i.split("?")[1] for i in data['features']]
qa_mem = QAMemory('question')
qa_mem.load_qa_to_memory(ques, ans)
agent = AgentBot(MODEL, qa_mem.memory)

Agent is ready


In [4]:
a = agent.agent("How can I recycle my product")
a



> Entering new  chain...
 I should consider if the product is recyclable
Action: assess_usage
Action Input: is the product recyclable

> Entering new  chain...
Prompt after formatting:
You are a secondhand dealer and assessing the user's product. Based on your questions and user answers from the chat history.
 AI: What is the screen size
Human:  2.8 inches
AI: What is the RAM size
Human:  512 MB
AI: What is the storage capacity
Human:  4 GB
AI: What is the battery capacity
Human:  1500 mAh
AI: Is there any malfunction or defect
Human:  yes
AI: What is the current physical condition of the product
Human:  excellent
AI: Is the product still under warranty
Human:  yes
 Please give your best answer for the given question from the user.
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"useable": string  // Is the given product still useable.
	"reason": string  // A reason why the product is

{'input': 'How can I recycle my product',
 'output': 'yes',
 'intermediate_steps': [(AgentAction(tool='assess_usage', tool_input='is the product recyclable', log=' I should consider if the product is recyclable\nAction: assess_usage\nAction Input: is the product recyclable'),
   '\n{\n  "useable": "yes",\n  "reason": "The product is made of recyclable materials.",\n  "function": "The product can function as well as new."\n}')]}

In [9]:
print(a.keys())
print(a['output'])

dict_keys(['input', 'output', 'intermediate_steps'])
yes


In [18]:
a['intermediate_steps'][0]

(AgentAction(tool='assess_usage', tool_input='is the product recyclable', log=' I should consider if the product is recyclable\nAction: assess_usage\nAction Input: is the product recyclable'),
 '\n{\n  "useable": "yes",\n  "reason": "The product is made of recyclable materials.",\n  "function": "The product can function as well as new."\n}')

In [26]:
print(a['intermediate_steps'][0][0])
print(a['intermediate_steps'][0][1])

AgentAction(tool='assess_usage', tool_input='is the product recyclable', log=' I should consider if the product is recyclable\nAction: assess_usage\nAction Input: is the product recyclable')

{
  "useable": "yes",
  "reason": "The product is made of recyclable materials.",
  "function": "The product can function as well as new."
}
